In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.linalg as la
import scipy.sparse as sparse
from matplotlib import cm
import timeit
import numba
from numba import jit

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
def u_analytical(x, y):

    uan = np.sin(np.pi*x)*np.sin(np.pi*y)

    return uan

def source(x, y):

    f = -2*np.pi**2*np.sin(np.pi*x)*np.sin(np.pi*y)

    return f

In [4]:
@jit(nopython=True)
def jacobi_iteration(matrix, source, init_guess = None, tolerance = 1.e-5, itermax = 200):

    M, f, u0, tol, kmax = matrix, source, init_guess, tolerance, itermax

    N = int(np.sqrt(M.shape[0])-1)
    # print(N)

    if u0 is None:

        u0 = np.zeros(f.shape)


    u = u0
    rel_diff = tol + 1
    k = 0

    while  k < kmax and rel_diff > tol:

        u_temp = np.zeros((N+1, N+1))

        for j in range(1, N):

            for i in range(1, N):
                
                u_temp[i, j] = (u[i-1, j] + u[i+1, j] + 
                                 u[i, j-1] + u[i, j+1] - f[i,j]*h**2)/4

                # print(M[i,i], M[i-1, j], M[i+1,j], M[i, j-1], M[i,j+1])

        rel_diff = la.norm(u_temp - u)/la.norm(u)
        
        # L2norm = la.norm(u_temp - u, 2)
        # maxnorm = la.norm(u_temp - u, np.inf)

        # print(abs_diff, L2norm, maxnorm)


        u = u_temp
        k += 1
        # print(u, '\n')
    return u, k, rel_diff

In [5]:
N = 40
Nx = N
Ny = N


x_I, x_F = 0, 1
y_I, y_F = 0, 1

h = (x_F - x_I)/N

X = np.linspace(x_I, x_F, N+1)
Y = np.linspace(x_I, x_F, N+1)

x, y = np.meshgrid(X, Y)

# print(h, '\n')
# print(h, '\n')


In [6]:
#creat main diagonal block
b1 = np.ones(Nx)
# b1[Nx-1] = 0

b2 = -4*np.ones(Nx+1)
# b2[0] = 1
# b2[Nx] = 1

b3 = np.ones(Nx)
# b3[0] = 0

B = sparse.diags([b1, b2, b3], [-1, 0, 1]).toarray()             #main block matrix

# print(B)

M1 = sparse.kron(sparse.eye(Ny+1), B).toarray()                  #central component of the tridiagonal block matrix

In [7]:
#creat upper and lower diagonal blocks
a = sparse.diags([np.ones(Ny), np.ones(Ny)], [-1, 1]).toarray()

A = sparse.eye(Nx+1).toarray()                                #upper/lower block matrix
# A[0, 0] = 0
# A[Nx, Nx] = 0

M2 = sparse.kron(a, A).toarray()                              #upper/lower component of the tridiagonal block matrix
# print(A)

In [8]:
#creat the tridiagonal block matrix of the 2d poisson equation.
M = M1 + M2       
print(M)


[[-4.  1.  0. ...  0.  0.  0.]
 [ 1. -4.  1. ...  0.  0.  0.]
 [ 0.  1. -4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -4.  1.  0.]
 [ 0.  0.  0. ...  1. -4.  1.]
 [ 0.  0.  0. ...  0.  1. -4.]]


In [9]:
#Test the solver
f = source(x, y)                          #source term
u_guess = np.sin(np.pi*x/2)*np.sin(np.pi*y/2)             #initial guess
itermax = 2000000                            #maximum number of iterations
tol = 1e-8                               #desired tolerance

start = timeit.default_timer()            #time the solver

u, iternum, rel_diff = jacobi_iteration(M, f, init_guess = u_guess, itermax = itermax, tolerance = tol)       #calculate the solution

end = timeit.default_timer()

elapsed = (end - start)

u_an = u_analytical(x, y)                #analytical solution for comparison

if iternum == itermax:

    print('WARNING: desired tolerance has not been reached for the given maximum iterations \n')

print('Number of iterations: {} \n'
      'L2 norm between last two iterations: {:1.2e} \n'
      'Time elapsed: {:1.2e} s'.format(iternum, rel_diff, elapsed))



Number of iterations: 3681 
L2 norm between last two iterations: 1.00e-08 
Time elapsed: 1.69e+00 s


In [15]:
u_an = u_analytical(x, y)

error = la.norm(u - u_an, 2)/la.norm(u_an, 2)

print(error)

0.0005109663257388609


In [10]:
#plot and compare the results
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


scat = ax.scatter(x, y, u, c = 'r', label = 'numerical', alpha = 1)
surf = ax.plot_surface(x, y, u_an, cmap = cm.coolwarm, label = 'analytical', alpha = 0.8)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('$u = \sin\pi x \sin\pi y$')

surf._facecolors2d=surf._facecolors3d
surf._edgecolors2d=surf._edgecolors3d
ax.legend(loc = 2)
plt.show()

In [11]:
# np.allclose(M.dot(u_an[1:N, 1:N].reshape((N-1)**2)), f[1:N, 1:N].reshape((N-1)**2)) 